<a href="https://colab.research.google.com/github/yoonhero/nanoGPT/blob/master/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-03-11 14:14:39--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-03-11 14:14:39 (20.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# string to index
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello"))
print(decode(encode("wow")))

[46, 43, 50, 50, 53]
wow


In [9]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'context: {context} target: {target}')

context: tensor([18]) target: 47
context: tensor([18, 47]) target: 56
context: tensor([18, 47, 56]) target: 57
context: tensor([18, 47, 56, 57]) target: 58
context: tensor([18, 47, 56, 57, 58]) target: 1
context: tensor([18, 47, 56, 57, 58,  1]) target: 15
context: tensor([18, 47, 56, 57, 58,  1, 15]) target: 47
context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) target: 58


In [14]:
batch_size = 4
block_size = 8

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print("------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

torch.Size([4, 8])
tensor([[45,  1, 25, 53, 52, 58, 39, 45],
        [51, 63,  1, 57, 61, 53, 56, 42],
        [61,  1,  5, 44, 53, 56,  1, 58],
        [54, 39, 57, 57,  1, 57, 61, 43]])
torch.Size([4, 8])
tensor([[ 1, 25, 53, 52, 58, 39, 45, 59],
        [63,  1, 57, 61, 53, 56, 42,  6],
        [ 1,  5, 44, 53, 56,  1, 58, 46],
        [39, 57, 57,  1, 57, 61, 43, 43]])
------
when input is [45] the target: 1
when input is [45, 1] the target: 25
when input is [45, 1, 25] the target: 53
when input is [45, 1, 25, 53] the target: 52
when input is [45, 1, 25, 53, 52] the target: 58
when input is [45, 1, 25, 53, 52, 58] the target: 39
when input is [45, 1, 25, 53, 52, 58, 39] the target: 45
when input is [45, 1, 25, 53, 52, 58, 39, 45] the target: 59
when input is [51] the target: 63
when input is [51, 63] the target: 1
when input is [51, 63, 1] the target: 57
when input is [51, 63, 1, 57] the target: 61
when input is [51, 63, 1, 57, 61] the target: 53
when input is [51, 63, 1, 57, 61, 5

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each toekn directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integes
        # C is the Channel which represents the embedding table output size
        # when we pass the idx to the token embedding table 
        # we get a embedidng tensor by the idx and get by one by one
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequnce
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss.item())
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
4.446467876434326

tB!!FgBr:&CiyVGX
w?,gMUtbAH
;Tn.?JWYN&,QT.rw&EZGBm!V.t;X.'J:$iQTq-cgFrpAn-?ZsQtvVuWFO,aN$cTnsCx'qiSg


In [31]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-2)

In [44]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5274710655212402


In [46]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=400)[0].tolist()))


Aurnda

ICLLIry t Whast

Y l chi'l NI s doronanen,

NTUCatealve CHer ckedot,
A mutiavan'IDUS t relitof oule terefertin.
Wou? n t gr m d goncouchas an wofll kinlath end fthain, saspo,
S: s ct w, dicalas.
Thyofourwhepiomue REWit whioubor be nouededgoum,
An s, he, spourswam matonoral yoroooue fasofonglon:
ATAnoutok an ICENTofowomy

Es flld rmenin m:

TIn igin bylled Fexthiche fr, s: I he yreay th pey
